In [1]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

from cv_vis import plot_training_stats

***AlexNet***

This is a simple analysis of general features of a simplified implementation (see NNs/AlexNet.py and runtime parameters in globals.py and cv.py) of the image recognition network AlexNet (Krizhevsky, Sutskever, and Hinton 2012).

AlexNet is a convolutional neural network with four max-pooled convolutional layers followed by two dense layers. For original structure, see Fig. 2 of the original paper (reproduced here). It is best known for being the first deep neural network to outperform traditional machine learning tools in the ImageNet challenge (image-net.org), in 2012.

In the following example, we will be classifying two classes: cats vs dogs from ImageNet, totalling 1127 and 791 pictures, respectively.

Before feeding to the network, the images were preprocessed with preprocess.py:
 - the 'No image available' downloads were discarded,
 - image was transformed into gray scale,
 - and it was resized to 256x256 (set by globals.py), with padding used to respect the original aspect ratio.
 No further standardization of the color scale was performed. Note that preprocessing was performed using tensorflow, and so can use GPU processing as available (in present version, however, only on image-by-image basis...).
 
To make the problem tractable on a laptop and avoid overfitting (after all, we are only classifying two classes), all dimension of the original AlexNet were divided the downscale parameter.

Lets start with a bare version of AlexNet. No dropout is used, downscale=16. The network is trained with Adam for 50 epochs:

In [5]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

from cv_vis import plot_training_stats
iplot(plot_training_stats("/DATA/Dropbox/LOOTRPV/Personal_programming/MachineLearning/Tutorials/ImageRecognition/workspace/AlexNet_zero.log"))